<a href="https://colab.research.google.com/github/kmk4444/System_engineering/blob/main/Ragas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://docs.ragas.io/en/v0.1.21/concepts/metrics/answer_relevance.html

In [ ]:
pip install langchain_openai ragas

In [ ]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from ragas import evaluate, EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import (
    AnswerRelevancy,
    Faithfulness,
    FactualCorrectness,
    AnswerCorrectness
)

# OpenAI API Anahtarını Ayarla
os.environ["OPENAI_API_KEY"] = "sk"

# Excel dosyasını oku (B sütununda sorular, C sütununda referans cevaplar, D sütununda deepseek-r1 cevapları)
df = pd.read_excel("/content/Veri Karşılaştırma_test.xlsx")  # Dosya yolunuza göre güncelleyiniz

# Kolonları uygun değişkenlere alalım
questions = df.iloc[:, 1].tolist()
reference_answers = df.iloc[:, 2].tolist()
deepseek_answers = df.iloc[:, 3].tolist()

# LLM Modelini Başlat
llm = ChatOpenAI(model="gpt-4o")  # GPT-4o değerlendirme için kullanılacak
evaluator_llm = LangchainLLMWrapper(llm)

# Değerlendirme için veri setini oluştur
dataset = []
for question, deepseek_response, reference in zip(questions, deepseek_answers, reference_answers):
    dataset.append(
        {
            "user_input": question,
            "retrieved_contexts": [],  # Bağlam kullanmadığımız için boş liste
            "response": deepseek_response,
            "reference": reference
        }
    )

# Dataset'i Ragas ile formatla
evaluation_dataset = EvaluationDataset.from_list(dataset)

# Ragas değerlendirmesini gerçekleştir
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[
        AnswerRelevancy(),
        Faithfulness(),
        FactualCorrectness(),
        AnswerCorrectness()
    ],
    llm=evaluator_llm
)

# Sonuçları yazdır
print("\n--- DeepSeek-r1 Değerlendirme Sonuçları (ChatGPT-4o Referans Olarak Kullanıldı) ---\n")
print(result)


Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]


--- DeepSeek-r1 Değerlendirme Sonuçları (ChatGPT-4o Referans Olarak Kullanıldı) ---

{'answer_relevancy': 0.8458, 'faithfulness': 0.0200, 'factual_correctness(mode=f1)': 0.2492, 'answer_correctness': 0.4218}


In [ ]:
# İlk 3 elemanı yazdırma
print("Questions - İlk 3 Satır:", questions[:3])
print("Reference Answers - İlk 3 Satır:", reference_answers[:3])
print("DeepSeek Answers - İlk 3 Satır:", deepseek_answers[:3])

# Son 3 elemanı yazdırma
print("\nQuestions - Son 3 Satır:", questions[-3:])
print("Reference Answers - Son 3 Satır:", reference_answers[-3:])
print("DeepSeek Answers - Son 3 Satır:", deepseek_answers[-3:])


Questions - İlk 3 Satır: ['Sistem mühendisliği nedir ve temel amacı nedir?', 'V-Model (V-Şeması) nedir ve hangi aşamaları içerir?', '“Sistem gereksinimleri” ile “kullanıcı gereksinimleri” arasındaki fark nedir?']
Reference Answers - İlk 3 Satır: ['Sistem mühendisliği, karmaşık sistemlerin tasarım, geliştirme, entegrasyon ve yaşam döngüsü yönetimiyle ilgilenen disiplinler arası bir alanıdır. Temel amacı, sistemleri en verimli şekilde tasarlamak, sürekliliğini sağlamak ve uçtan uca yönetmektir.', 'V-Model, sistem veya yazılım geliştirme yaşam döngüsünü tanımlamak için kullanılan bir modeldir. Sol kolunda gereksinim analizi ve tasarım faaliyetleri, sağ kolunda ise test ve doğrulama faaliyetleri bulunur. Ortada entegrasyon aşaması yer alır ve süreç, gereksinimlerden doğrulamaya kadar sistematik bir biçimde ilerler.', 'Sistem gereksinimleri, bir sistemin teknik ve işlevsel niteliklerini tanımlar ve tasarım ekibi tarafından anlaşılması gereken ayrıntıları içerir. Kullanıcı gereksinimleri ise

In [ ]:
pip install "trulens-eval[openai,langchain]" --upgrade

In [ ]:
# Eski kırıntıları temizle (opsiyonel ama tavsiye edilir)
!pip uninstall -y trulens trulens-eval trulens-providers-openai trulens-apps-langchain

In [ ]:
# Gerekli her şeyi tek seferde yükle
!pip install "trulens-eval==1.4.9" trulens-providers-openai trulens-apps-langchain langchain-community

In [ ]:
#   1) Köprü (back-compat) yolu
from trulens_eval.feedback.provider.openai import OpenAI as TruOpenAI
from trulens_eval import TruBasicApp, Feedback, Select, Tru

#   2) Doğrudan yeni modüler yol
from trulens.providers.openai import OpenAI as TruOpenAI
from trulens.apps.langchain import TruChain          # gerekiyorsa
#from trulens import Tru, Feedback, Select
import pandas as pd

In [ ]:
# --- 1. Veri setini oku -----------------------------------------------------
df = pd.read_excel("/content/Data karşılaştırma.xlsx")

questions         = df.iloc[:, 1].tolist()   # B sütunu
reference_answers = df.iloc[:, 2].tolist()   # C sütunu
model_answers     = df.iloc[:, 3].tolist()   # D sütunu

# --- 2. Hakem LLM ve provider ----------------------------------------------
judge_llm = ChatOpenAI(model="gpt-4o", temperature=0)
provider  = TruOpenAI(llm=judge_llm)

# --- 3. Feedback tanımları --------------------------------------------------
f_relevance = (
    Feedback(
        AnswerRelevance(provider=provider).measure,
        name="Answer Relevance"
    )
    .on_input_output()
)

f_grounded = (
    Feedback(
        Groundedness(provider=provider).measure,
        name="Faithfulness/Groundedness"
    )
    .on_input_output()
)

# Referans cevaba yakınlık: referansı her kaydın inputs'undan çekeceğiz
f_gt_agree = (
    Feedback(
        GroundTruthAgreement(provider=provider).agreement_measure,
        name="Ground-Truth Agreement"
    )
    .on_output()                                 # model cevabı
    .against(lambda rec: rec.inputs["reference"])  # altın cevap
)

# --- 4. Basit Tru app -------------------------------------------------------
tru_app = TruBasicApp(
    app_id   = "deepseek_eval",
    feedbacks= [f_relevance, f_grounded, f_gt_agree]
)

# --- 5. Kayıt döngüsü -------------------------------------------------------
tru = Tru()
tru.reset_database()

for q, ref, ans in zip(questions, reference_answers, model_answers):
    tru_app.record(
        inputs  = {"prompt": q, "reference": ref},
        outputs = {"response": ans}
    )

# --- 6. Sonuçları çek -------------------------------------------------------
records = tru.get_records(app_ids=["deepseek_eval"])

for rec in records:
    print("Soru :", rec.record.inputs["prompt"][:80])
    print("Cevap:", rec.record.outputs["response"][:80])
    for fb in rec.feedback:
        print(f"  {fb.name}: {fb.value:.2f}")
    print("-"*60)


## Sample by Sample Examination Chatgpt



In [ ]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas import evaluate, EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import (
    AnswerRelevancy,
    Faithfulness,
    FactualCorrectness,
    AnswerCorrectness
)

# === 1. OpenAI anahtarı ===
os.environ["OPENAI_API_KEY"] = "sk--"  # kendi anahtarını yaz

# === 2. Excel dosyasını oku ===
df = pd.read_excel("/content/Veri Karşılaştırma_test.xlsx", sheet_name="Chatgpt")

# === 3. Değerlendirici modeller ===
llm = ChatOpenAI(model="gpt-4o", temperature=0)
evaluator_llm = LangchainLLMWrapper(llm)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# === 4. Model sütunları ===
model_columns = {
    "deepseek-r1": "Deepseek-r1 (System chatbot)",
    "llama-3.1": "Llama 3.1 8b (System Chatbot)",
    "claude-3.7": "Claude Cevap"
}

# === 5. Her modelin skorlarını hesapla ===
all_results = []

for model_name, col_name in model_columns.items():
    print(f"\n--- {model_name} modeli değerlendiriliyor ---")

    dataset_list = []
    for i, row in df.iterrows():
        dataset_list.append({
            "user_input": str(row["Soru"]),
            "retrieved_contexts": [],  # RAG bağlamı yok
            "response": str(row[col_name]),
            "reference": str(row["Cevap-Chatgpt-o1"])
        })

    evaluation_dataset = EvaluationDataset.from_list(dataset_list)

    result = evaluate(
        dataset=evaluation_dataset,
        metrics=[
            AnswerRelevancy(),
            Faithfulness(),
            FactualCorrectness(),
            AnswerCorrectness()
        ],
        llm=evaluator_llm,
        embeddings=embeddings
    )

    # örnek bazlı skorları yakala
    if hasattr(result, "scores"):
        df_scores = pd.DataFrame(result.scores)
    elif hasattr(result, "to_pandas"):
        df_scores = result.to_pandas()
    elif isinstance(result, dict) and "results" in result:
        df_scores = pd.DataFrame(result["results"])
    else:
        raise ValueError("Skor verisi bulunamadı")

    df_scores["model"] = model_name
    df_scores["dataset"] = "chatgpt"
    df_scores["sample_id"] = df["Soru No"]

    all_results.append(df_scores)

# === 6. Tek dosya olarak kaydet ===
final_df = pd.concat(all_results, ignore_index=True)
final_df.rename(columns={
    "answer_relevancy": "AnswerRelevancy",
    "faithfulness": "Faithfulness",
    "factual_correctness": "FactualCorrectness",
    "answer_correctness": "AnswerCorrectness"
}, inplace=True)

output_path = "/content/ragas_per_sample_chatgpt.xlsx"
final_df.to_excel(output_path, index=False)
print(f"\n✅ Kaydedildi: {output_path}")

from google.colab import files
files.download(output_path)


## Sample by Sample Examination Chatgpt



In [10]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas import evaluate, EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import (
    AnswerRelevancy,
    Faithfulness,
    FactualCorrectness,
    AnswerCorrectness
)

# === 1. OpenAI anahtarı ===
os.environ["OPENAI_API_KEY"] = "sk-proj--"  # kendi anahtarını yaz

# === 2. Excel dosyasını oku ===
df = pd.read_excel("/content/Veri Karşılaştırma_test.xlsx", sheet_name="Gemini")

# === 3. Değerlendirici modeller ===
llm = ChatOpenAI(model="gpt-4o", temperature=0)
evaluator_llm = LangchainLLMWrapper(llm)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# === 4. Model sütunları ===
model_columns = {
    "deepseek-r1": "Deepseek-r1 (System chatbot)",
    "llama-3.1": "Llama 3.1 8b (System Chatbot)",
    "claude-3.7": "Claude Cevap"
}

# === 5. Her modelin skorlarını hesapla ===
all_results = []

for model_name, col_name in model_columns.items():
    print(f"\n--- {model_name} modeli değerlendiriliyor ---")

    dataset_list = []
    for i, row in df.iterrows():
        dataset_list.append({
            "user_input": str(row["Soru"]),
            "retrieved_contexts": [],  # RAG bağlamı yok
            "response": str(row[col_name]),
            "reference": str(row["Cevap-Gemini 2.0 Flash"])
        })

    evaluation_dataset = EvaluationDataset.from_list(dataset_list)

    result = evaluate(
        dataset=evaluation_dataset,
        metrics=[
            AnswerRelevancy(),
            Faithfulness(),
            FactualCorrectness(),
            AnswerCorrectness()
        ],
        llm=evaluator_llm,
        embeddings=embeddings
    )

    # örnek bazlı skorları yakala
    if hasattr(result, "scores"):
        df_scores = pd.DataFrame(result.scores)
    elif hasattr(result, "to_pandas"):
        df_scores = result.to_pandas()
    elif isinstance(result, dict) and "results" in result:
        df_scores = pd.DataFrame(result["results"])
    else:
        raise ValueError("Skor verisi bulunamadı")

    df_scores["model"] = model_name
    df_scores["dataset"] = "gemini"
    df_scores["sample_id"] = df["Soru No"]

    all_results.append(df_scores)

# === 6. Tek dosya olarak kaydet ===
final_df = pd.concat(all_results, ignore_index=True)
final_df.rename(columns={
    "answer_relevancy": "AnswerRelevancy",
    "faithfulness": "Faithfulness",
    "factual_correctness": "FactualCorrectness",
    "answer_correctness": "AnswerCorrectness"
}, inplace=True)

output_path = "/content/ragas_per_sample_gemini.xlsx"
final_df.to_excel(output_path, index=False)
print(f"\n✅ Kaydedildi: {output_path}")

from google.colab import files
files.download(output_path)


/tmp/ipython-input-939793228.py:22: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  evaluator_llm = LangchainLLMWrapper(llm)



--- deepseek-r1 modeli değerlendiriliyor ---


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]


--- llama-3.1 modeli değerlendiriliyor ---


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]


--- claude-3.7 modeli değerlendiriliyor ---


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]


✅ Kaydedildi: /content/ragas_per_sample_gemini.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## WILCOXON ANLAMLILIK ANALİZİ (ChatGPT + Gemini) (Simple)

In [13]:
import pandas as pd
from scipy.stats import wilcoxon
import numpy as np

# === 1️⃣ İki veri setini yükle ===
chatgpt_df = pd.read_excel("/content/ragas_per_sample_chatgpt.xlsx")
gemini_df = pd.read_excel("/content/ragas_per_sample_gemini.xlsx")

# === 2️⃣ Birleştir ===
df = pd.concat([chatgpt_df, gemini_df], ignore_index=True)
print("Veri setleri birleştirildi:", df.shape)

# === 3️⃣ Metrik sütunlarını normalize et (isim farklarını düzelt) ===
df.rename(columns={
    "answer_relevancy": "AnswerRelevancy",
    "faithfulness": "Faithfulness",
    "factual_correctness(mode=f1)": "FactualCorrectness",
    "factual_correctness": "FactualCorrectness",
    "answer_correctness": "AnswerCorrectness"
}, inplace=True)

# === 4️⃣ Wilcoxon testi için fonksiyon ===
def wilcoxon_compare(df, dataset, metric, model_a, model_b):
    sub = df[df["dataset"] == dataset]
    a = sub[sub["model"] == model_a][metric].astype(float).reset_index(drop=True)
    b = sub[sub["model"] == model_b][metric].astype(float).reset_index(drop=True)
    n = min(len(a), len(b))
    if n < 2:
        return (np.nan, np.nan, n)
    stat, p = wilcoxon(a[:n], b[:n])
    return (stat, p, n)

# === 5️⃣ Model çiftleri ve metrikler ===
model_pairs = [
    ("llama-3.1", "deepseek-r1"),
    ("claude-3.7", "llama-3.1"),
    ("claude-3.7", "deepseek-r1")
]
metrics = ["AnswerRelevancy", "Faithfulness", "FactualCorrectness", "AnswerCorrectness"]
datasets = ["chatgpt", "gemini"]

# === 6️⃣ Testleri çalıştır ===
rows = []
for ds in datasets:
    for met in metrics:
        for (mA, mB) in model_pairs:
            try:
                stat, p, n = wilcoxon_compare(df, ds, met, mA, mB)
                sig = "✅ anlamlı" if p < 0.05 else "❌ anlamsız"
                rows.append({
                    "dataset": ds,
                    "metric": met,
                    "model_A": mA,
                    "model_B": mB,
                    "W": stat,
                    "p_value": p,
                    "n_pairs": n,
                    "result": sig
                })
            except Exception as e:
                rows.append({
                    "dataset": ds,
                    "metric": met,
                    "model_A": mA,
                    "model_B": mB,
                    "error": str(e)
                })

results = pd.DataFrame(rows)
results.to_excel("/content/wilcoxon_results.xlsx", index=False)
print("✅ Wilcoxon analiz tamamlandı. Sonuçlar kaydedildi: wilcoxon_results.xlsx")

from google.colab import files
files.download("/content/wilcoxon_results.xlsx")

results.head(10)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,dataset,metric,model_A,model_B,W,p,median_delta,rank_biserial_r,effect_size,p_holm,result
0,chatgpt,AnswerRelevancy,llama-3.1,deepseek-r1,6.0,0.8125,0.036130,0.600000,büyük,1.0,❌ anlamsız
1,chatgpt,AnswerRelevancy,claude-3.7,llama-3.1,6.0,0.8125,0.100401,0.600000,büyük,1.0,❌ anlamsız
2,chatgpt,AnswerRelevancy,claude-3.7,deepseek-r1,1.0,0.5000,0.000000,0.933333,büyük,1.0,❌ anlamsız
3,chatgpt,Faithfulness,llama-3.1,deepseek-r1,4.0,0.8750,0.093333,0.733333,büyük,1.0,❌ anlamsız
4,chatgpt,Faithfulness,claude-3.7,llama-3.1,5.0,1.0000,0.000000,0.666667,büyük,1.0,❌ anlamsız
5,chatgpt,Faithfulness,claude-3.7,deepseek-r1,0.0,1.0000,0.000000,1.000000,büyük,1.0,❌ anlamsız
6,chatgpt,FactualCorrectness,llama-3.1,deepseek-r1,7.0,1.0000,-0.110000,0.533333,büyük,1.0,❌ anlamsız
7,chatgpt,FactualCorrectness,claude-3.7,llama-3.1,6.0,0.8125,-0.170000,0.600000,büyük,1.0,❌ anlamsız
8,chatgpt,FactualCorrectness,claude-3.7,deepseek-r1,4.0,0.8750,0.080000,0.733333,büyük,1.0,❌ anlamsız
9,chatgpt,AnswerCorrectness,llama-3.1,deepseek-r1,7.0,1.0000,0.005995,0.533333,büyük,1.0,❌ anlamsız


## WILCOXON ANLAMLILIK ANALİZİ (ChatGPT + Gemini) (Detail)

In [14]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests

# === Verileri yükle ===
chatgpt_df = pd.read_excel("/content/ragas_per_sample_chatgpt.xlsx")
gemini_df = pd.read_excel("/content/ragas_per_sample_gemini.xlsx")
df = pd.concat([chatgpt_df, gemini_df], ignore_index=True)

# === Kolon adlarını normalize et ===
df.rename(columns={
    "answer_relevancy": "AnswerRelevancy",
    "faithfulness": "Faithfulness",
    "factual_correctness(mode=f1)": "FactualCorrectness",
    "factual_correctness": "FactualCorrectness",
    "answer_correctness": "AnswerCorrectness"
}, inplace=True)

# === Wilcoxon fonksiyonu ===
def run_wilcoxon(df, dataset, metric, model_a, model_b):
    sub = df[df["dataset"] == dataset]
    a = sub[sub["model"] == model_a][metric].astype(float).reset_index(drop=True)
    b = sub[sub["model"] == model_b][metric].astype(float).reset_index(drop=True)
    n = min(len(a), len(b))
    if n < 5:  # küçük örneklem kontrolü
        return None
    # farklar
    diffs = a[:n] - b[:n]
    # Wilcoxon testi
    stat, p = wilcoxon(a[:n], b[:n])
    # Etki büyüklüğü (rank biserial)
    rank_r = 1 - (2 * stat) / (n * (n + 1))
    # Medyan fark
    median_delta = np.median(diffs)
    # Etki boyutu yorumu
    if abs(rank_r) < 0.3:
        eff = "küçük"
    elif abs(rank_r) < 0.5:
        eff = "orta"
    else:
        eff = "büyük"
    return {"dataset": dataset, "metric": metric, "model_A": model_a, "model_B": model_b,
            "W": stat, "p": p, "median_delta": median_delta,
            "rank_biserial_r": rank_r, "effect_size": eff}

# === Model çiftleri, metrikler ===
model_pairs = [
    ("llama-3.1", "deepseek-r1"),
    ("claude-3.7", "llama-3.1"),
    ("claude-3.7", "deepseek-r1")
]
metrics = ["AnswerRelevancy", "Faithfulness", "FactualCorrectness", "AnswerCorrectness"]
datasets = ["chatgpt", "gemini"]

# === Tüm testleri çalıştır ===
results = []
for ds in datasets:
    for met in metrics:
        for (mA, mB) in model_pairs:
            out = run_wilcoxon(df, ds, met, mA, mB)
            if out:
                results.append(out)

res_df = pd.DataFrame(results)

# Holm-Bonferroni düzeltmesi
res_df["p_holm"] = multipletests(res_df["p"], method="holm")[1]
res_df["result"] = np.where(res_df["p_holm"] < 0.05, "✅ anlamlı", "❌ anlamsız")

# Kaydet
res_df.to_excel("/content/wilcoxon_advanced.xlsx", index=False)
from google.colab import files
files.download("/content/wilcoxon_advanced.xlsx")

res_df.head(10)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,dataset,metric,model_A,model_B,W,p,median_delta,rank_biserial_r,effect_size,p_holm,result
0,chatgpt,AnswerRelevancy,llama-3.1,deepseek-r1,6.0,0.8125,0.036130,0.600000,büyük,1.0,❌ anlamsız
1,chatgpt,AnswerRelevancy,claude-3.7,llama-3.1,6.0,0.8125,0.100401,0.600000,büyük,1.0,❌ anlamsız
2,chatgpt,AnswerRelevancy,claude-3.7,deepseek-r1,1.0,0.5000,0.000000,0.933333,büyük,1.0,❌ anlamsız
3,chatgpt,Faithfulness,llama-3.1,deepseek-r1,4.0,0.8750,0.093333,0.733333,büyük,1.0,❌ anlamsız
4,chatgpt,Faithfulness,claude-3.7,llama-3.1,5.0,1.0000,0.000000,0.666667,büyük,1.0,❌ anlamsız
5,chatgpt,Faithfulness,claude-3.7,deepseek-r1,0.0,1.0000,0.000000,1.000000,büyük,1.0,❌ anlamsız
6,chatgpt,FactualCorrectness,llama-3.1,deepseek-r1,7.0,1.0000,-0.110000,0.533333,büyük,1.0,❌ anlamsız
7,chatgpt,FactualCorrectness,claude-3.7,llama-3.1,6.0,0.8125,-0.170000,0.600000,büyük,1.0,❌ anlamsız
8,chatgpt,FactualCorrectness,claude-3.7,deepseek-r1,4.0,0.8750,0.080000,0.733333,büyük,1.0,❌ anlamsız
9,chatgpt,AnswerCorrectness,llama-3.1,deepseek-r1,7.0,1.0000,0.005995,0.533333,büyük,1.0,❌ anlamsız
